# Classification with TF Decision Trees
Source code from https://keras.io/examples/structured_data/classification_with_tfdf/

In [ ]:
!pip install huggingface_hub

In [ ]:
!pip install -U tensorflow_decision_forests

In [ ]:
!pip install tensorflow==2.7.0

In [ ]:
!pip install ipykernel==4.10

In [ ]:
!apt-get install -y git-lfs

In [ ]:
from huggingface_hub import notebook_login
from huggingface_hub import push_to_hub_keras

notebook_login()

In [79]:
import math
import urllib
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_decision_forests as tfdf


In [80]:
input_path = "https://archive.ics.uci.edu/ml/machine-learning-databases/census-income-mld/census-income"
input_column_header = "income_level"


In [81]:
#Load data

BASE_PATH = input_path
CSV_HEADER = [ l.decode("utf-8").split(":")[0].replace(" ", "_")
  for l in urllib.request.urlopen(f"{BASE_PATH}.names")
  if not l.startswith(b"|")][2:]

CSV_HEADER.append(input_column_header)

train_data = pd.read_csv(f"{BASE_PATH}.data.gz", header=None, names=CSV_HEADER)
test_data = pd.read_csv(f"{BASE_PATH}.test.gz", header=None, names=CSV_HEADER)

In [ ]:
print(train_data.head)

In [ ]:
#convert from string to integers
target_labels = [" - 50000.", " 50000+."]
train_data[input_column_header] = train_data[input_column_header].map(target_labels.index)
test_data[input_column_header] = test_data[input_column_header].map(target_labels.index)

In [ ]:
#Observe shape of training and test data
print(f"Train data shape: {train_data.shape}")
print(f"Test data shape: {test_data.shape}")
print(train_data.head().T)

In [12]:
#define metadata

# Target column name.
TARGET_COLUMN_NAME = "income_level"
# Weight column name.
WEIGHT_COLUMN_NAME = "instance_weight"
# Numeric feature names.
NUMERIC_FEATURE_NAMES = [
    "age",
    "wage_per_hour",
    "capital_gains",
    "capital_losses",
    "dividends_from_stocks",
    "num_persons_worked_for_employer",
    "weeks_worked_in_year",
]

# Categorical features and their vocabulary lists.
CATEGORICAL_FEATURES_WITH_VOCABULARY = {
    feature_name: sorted(
        [str(value) for value in list(train_data[feature_name].unique())]
    )
    for feature_name in CSV_HEADER
    if feature_name
    not in list(NUMERIC_FEATURE_NAMES + [WEIGHT_COLUMN_NAME, TARGET_COLUMN_NAME])
}
# All features names.
FEATURE_NAMES = NUMERIC_FEATURE_NAMES + list(
    CATEGORICAL_FEATURES_WITH_VOCABULARY.keys()
)

Configure hyperparameters for the tree model.

In [13]:
GROWING_STRATEGY = "BEST_FIRST_GLOBAL"
NUM_TREES = 250
MIN_EXAMPLES = 6
MAX_DEPTH = 5
SUBSAMPLE = 0.65
SAMPLING_METHOD = "RANDOM"
VALIDATION_RATIO = 0.1

In [55]:
#Implement training & evaluation procedure
def prepare_sample(features, target, weight):
    for feature_name in features:
        if feature_name in CATEGORICAL_FEATURES_WITH_VOCABULARY:
            if features[feature_name].dtype != tf.dtypes.string:
                # Convert categorical feature values to string.
                features[feature_name] = tf.strings.as_string(features[feature_name])
    return features, target, weight


def run_experiment(model, train_data, test_data, num_epochs=1, batch_size=None):

    train_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(
        train_data, label=TARGET_COLUMN_NAME, weight=WEIGHT_COLUMN_NAME
    ).map(prepare_sample, num_parallel_calls=tf.data.AUTOTUNE)
    test_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(
        test_data, label=TARGET_COLUMN_NAME, weight=WEIGHT_COLUMN_NAME
    ).map(prepare_sample, num_parallel_calls=tf.data.AUTOTUNE)

    model.fit(train_dataset, epochs=num_epochs, batch_size=batch_size)
    _, accuracy = model.evaluate(test_dataset, verbose=0)
    push_to_hub = True
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    

In [56]:
#Create model inputs

def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        if feature_name in NUMERIC_FEATURE_NAMES:
            inputs[feature_name] = layers.Input(
                name=feature_name, shape=(), dtype=tf.float32
            )
        else:
            inputs[feature_name] = layers.Input(
                name=feature_name, shape=(), dtype=tf.string
            )
    return inputs

# Experiment 1: Decision Forests with raw features

In [57]:
#Decision Forest with raw features
def specify_feature_usages(inputs):
    feature_usages = []

    for feature_name in inputs:
        if inputs[feature_name].dtype == tf.dtypes.float32:
            feature_usage = tfdf.keras.FeatureUsage(
                name=feature_name, semantic=tfdf.keras.FeatureSemantic.NUMERICAL
            )
        else:
            feature_usage = tfdf.keras.FeatureUsage(
                name=feature_name, semantic=tfdf.keras.FeatureSemantic.CATEGORICAL
            )

        feature_usages.append(feature_usage)
    return feature_usages
  

In [82]:
#Create GB trees model
def create_gbt_model():
    gbt_model = tfdf.keras.GradientBoostedTreesModel(
        features = specify_feature_usages(create_model_inputs()),
        exclude_non_specified_features = True,
        growing_strategy = GROWING_STRATEGY,
        num_trees = NUM_TREES,
        max_depth = MAX_DEPTH,
        min_examples = MIN_EXAMPLES,
        subsample = SUBSAMPLE,
        validation_ratio = VALIDATION_RATIO,
        task = tfdf.keras.Task.CLASSIFICATION,
        loss = "DEFAULT",
    )

    gbt_model.compile(metrics=[keras.metrics.BinaryAccuracy(name="accuracy")])
    return gbt_model
    push_to_hub_keras(gbt_model, organization="keras-io", repo_url="tdubon/Classification_TF_Forest") 

In [94]:
#Train and evaluate model
gbt_model = create_gbt_model()
run_experiment(gbt_model, train_data, test_data)

Use /tmp/tmpnkjxn57q as temporary training directory


/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core.py:2036: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = dataframe.drop(label, 1)
/usr/local/lib/python3.7/dist-packages/tensorflow_decision_forests/keras/core.py:2039: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  features_dataframe = features_dataframe.drop(weight, 1)


Starting reading the dataset
199/200 [============================>.] - ETA: 0s
Dataset read in 0:00:07.569854
Training model
Model trained in 0:01:40.791314
Compiling model
200/200 [==============================] - 108s 542ms/step
Test accuracy: 95.79%


In [ ]:
#Inspect the model: Model type, mask, input features, feature importance
print(gbt_model.summary())

In [85]:
#plot the model
tfdf.model_plotter.plot_model_in_colab(gbt_model, tree_idx=0, max_depth=3)

In [86]:
inspector = gbt_model.make_inspector()
[field for field in dir(inspector) if not field.startswith("_")]

['MODEL_NAME',
 'bias',
 'dataspec',
 'evaluation',
 'export_to_tensorboard',
 'extract_all_trees',
 'extract_tree',
 'features',
 'header',
 'iterate_on_nodes',
 'label',
 'label_classes',
 'loss',
 'metadata',
 'model_type',
 'num_trees',
 'num_trees_per_iter',
 'objective',
 'specialized_header',
 'task',
 'training_logs',
 'variable_importances']

In [ ]:
#display variable importance
inspector.variable_importances()

In [91]:
inspector.features()

["age" (1; #0),
 "capital_gains" (1; #1),
 "capital_losses" (1; #2),
 "citizenship" (4; #3),
 "class_of_worker" (4; #4),
 "country_of_birth_father" (4; #5),
 "country_of_birth_mother" (4; #6),
 "country_of_birth_self" (4; #7),
 "detailed_household_and_family_stat" (4; #8),
 "detailed_household_summary_in_household" (4; #9),
 "detailed_industry_recode" (4; #10),
 "detailed_occupation_recode" (4; #11),
 "dividends_from_stocks" (1; #12),
 "education" (4; #13),
 "enroll_in_edu_inst_last_wk" (4; #14),
 "family_members_under_18" (4; #15),
 "fill_inc_questionnaire_for_veteran's_admin" (4; #16),
 "full_or_part_time_employment_stat" (4; #17),
 "hispanic_origin" (4; #18),
 "live_in_this_house_1_year_ago" (4; #19),
 "major_industry_code" (4; #20),
 "major_occupation_code" (4; #21),
 "marital_stat" (4; #22),
 "member_of_a_labor_union" (4; #23),
 "migration_code-change_in_msa" (4; #24),
 "migration_code-change_in_reg" (4; #25),
 "migration_code-move_within_reg" (4; #26),
 "migration_prev_res_in_sun

In [95]:
tf.keras.models.save_model(
    gbt_model, "/Users/tdubon/TFClassificationForest", overwrite=True, include_optimizer=True, save_format=None,
    signatures=None, options=None, save_traces=True)

ValueError: ignored